In [1]:
from pyspark.sql.types import *
from pyspark.sql import functions as F

list=[[['3927'],'Hey'],
      [['3410'],'Yo'],
      [['3927'],'Why'],
      [['2227','3410'],'Am'],
      [['3410','3927'],'Here']]
cSchema = StructType([StructField("Code", ArrayType(StringType())),StructField("Document_title", StringType())])
df= spark.createDataFrame(list,schema=cSchema)
df.show()

+------------+--------------+
 Code|Document_title|
+------------+--------------+
 [3927]| Hey|
 [3410]| Yo|
 [3927]| Why|
[2227, 3410]| Am|
[3410, 3927]| Here|
+------------+--------------+

In [2]:
df1=df.withColumn("Code",F.explode(F.col("Code")))
df1.groupBy(F.col("Code"))\
   .agg(F.collect_list("Document_title"))\
   .show()

+----+----------------------------+
Code|collect_list(Document_title)|
+----+----------------------------+
3927| [Hey, Why, Here]|
3410| [Yo, Am, Here]|
2227| [Am]|
+----+----------------------------+

In [3]:
%scala
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions.{avg,col,lit,when}
val df =List(("ABC", 10),
              ("ABC", 20),
              ("ABC", 30),
              ("ABC", 40),
              ("ABC", 50),
              ("ABC", 60),
              ("ABC", 70),
              ("ABC", 80),
              ("ABC", 90),
              ("ABC", 100)
            ).
          toDF("name", "value")

val window = Window.partitionBy($"name").orderBy($"value").rowsBetween(-2, -1)
val df2 = df.withColumn("rolling_average", avg($"value") over(window)).withColumn("rolling_average", when(col("rolling_average")===10, lit(null)).otherwise(col("rolling_average")))
df2.show()

+----+-----+---------------+
name|value|rolling_average|
+----+-----+---------------+
 ABC| 10| null|
 ABC| 20| null|
 ABC| 30| 15.0|
 ABC| 40| 25.0|
 ABC| 50| 35.0|
 ABC| 60| 45.0|
 ABC| 70| 55.0|
 ABC| 80| 65.0|
 ABC| 90| 75.0|
 ABC| 100| 85.0|
+----+-----+---------------+

import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions.{avg, col, lit, when}
df: org.apache.spark.sql.DataFrame = [name: string, value: int]
window: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@66c7eea0
df2: org.apache.spark.sql.DataFrame = [name: string, value: int ... 1 more field]

In [4]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

list=[['a',[[95.0], [25.0, 25.0], [40.0]]],
      ['a',[[95.0], [20.0, 80.0]]],
      ['a',[[95.0], [25.0, 75.0]]],
      ['b',[[95.0], [25.0, 75.0]]],
      ['b',[[95.0], [12.0, 88.0]]]]
      
cSchema = StructType([StructField("A", StringType())\
                      ,StructField("B", ArrayType(ArrayType(FloatType())))])
df= spark.createDataFrame(list,schema=cSchema)

df.printSchema()
df.show()

root
-- A: string (nullable = true)
-- B: array (nullable = true)
 |-- element: array (containsNull = true)
 | |-- element: float (containsNull = true)

+---+--------------------+
 A| B|
+---+--------------------+
 a|[[95.0], [25.0, 2...|
 a|[[95.0], [20.0, 8...|
 a|[[95.0], [25.0, 7...|
 b|[[95.0], [25.0, 7...|
 b|[[95.0], [12.0, 8...|
+---+--------------------+

In [5]:
def remove_highest(col):
    return (np.sort( np.asarray([item for sublist in col for item in sublist])  )[:-1]).tolist()

udf_remove_highest = F.udf( remove_highest , ArrayType(FloatType()) )
df.withColumn("B",udf_remove_highest("B")).show()

+---+------------------+
 A| B|
+---+------------------+
 a|[25.0, 25.0, 40.0]|
 a| [20.0, 80.0]|
 a| [25.0, 75.0]|
 b| [25.0, 75.0]|
 b| [12.0, 88.0]|
+---+------------------+

In [6]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

expression="""filter(B, x -> x != C )"""
df1=df.withColumn("B",(F.sort_array(F.flatten("B")))).withColumn("C",F.array_max("B")).withColumn("B", F.expr(expression) )\
.drop("C")
df1.show()


  
  

+---+------------------+
 A| B|
+---+------------------+
 a|[25.0, 25.0, 40.0]|
 a| [20.0, 80.0]|
 a| [25.0, 75.0]|
 b| [25.0, 75.0]|
 b| [12.0, 88.0]|
+---+------------------+

In [7]:
arr=[25.0,25.0,40.0,95.0]
arr[:-1]

Out[130]: [25.0, 25.0, 40.0]

In [8]:
df1.printSchema()

root
-- A: string (nullable = true)
-- B: array (nullable = true)
 |-- element: float (containsNull = true)
-- Max: struct (nullable = false)
 |-- col1: float (nullable = true)

In [9]:
df1=df.withColumn("B",(F.sort_array(F.flatten("B")))).withColumn("Max", F.array(F.array_max("B")))
df1.withColumn("B",F.array_except("B","Max")).drop("Max").show()

+---+------------+
 A| B|
+---+------------+
 a|[25.0, 40.0]|
 a|[20.0, 80.0]|
 a|[25.0, 75.0]|
 b|[25.0, 75.0]|
 b|[12.0, 88.0]|
+---+------------+

In [10]:
df1.withColumn("B", F.array_remove("B", F.col("Max"))).show()

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-3543595061519617> in <module> 
 ----> 1 df1 . withColumn ( "B" , F . array_remove ( "B" , F . col ( "Max" ) ) ) . show ( ) 

 /databricks/spark/python/pyspark/sql/functions.py in array_remove (col, element) 
 2012 """
 2013 sc = SparkContext . _active_spark_context
 -> 2014 return Column ( sc . _jvm . functions . array_remove ( _to_java_column ( col ) , element ) ) 
 2015 
 2016 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1246 
 1247 def __call__ ( self , * args ) : 
 -> 1248 args_command , temp_args = self . _build_args ( * args ) 
 1249 
 1250 command = proto . CALL_COMMAND_NAME + \ 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in _build_args (self, *args) 
 1210 def _build_args ( self , * args ) : 
 1211 if self . converters is not None and len ( self . converters ) > 0 : 
 -> 1212 ( new_args , temp_args ) = self . _get_args ( args ) 
 1213 else : 
 1214 new_args = args

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in _get_args (self, args) 
 1197 for converter in self . gateway_client . converters : 
 1198 if converter . can_convert ( arg ) : 
 -> 1199 temp_arg = converter . convert ( arg , self . gateway_client ) 
 1200 temp_args . append ( temp_arg ) 
 1201 new_args . append ( temp_arg ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_collections.py in convert (self, object, gateway_client) 
 498 ArrayList = JavaClass ( "java.util.ArrayList" , gateway_client ) 
 499 java_list = ArrayList ( ) 
 --> 500 for element in object : 
 501 java_list . add ( element ) 
 502 return java_list

 /databricks/spark/python/pyspark/sql/column.py in __iter__ (self) 
 342 
 343 def __iter__ ( self ) : 
 --> 344 raise TypeError ( "Column is not iterable" ) 
 345 
 346 # string methods 

 TypeError : Column is not iterable

In [11]:
array(struct(lit(None).cast("string").alias("value"), lit(0).alias("amount")))